### Acknowledgment
-[m5 baseline](https://www.kaggle.com/harupy/m5-baseline) by harupy

-[Back to (predict) the future - Interactive M5 EDA](https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda) by headsortails

-[123rd place solution](https://www.kaggle.com/xxbxyae/123rd-place-solution) by Kien

In [ ]:
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
register_matplotlib_converters()
sns.set()

In [ ]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
def read_data():
    INPUT_DIR = f"/kaggle/input/m5-forecasting-accuracy"

    calendar = pd.read_csv(f"{INPUT_DIR}/calendar.csv").pipe(reduce_mem_usage)
    prices = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv").pipe(reduce_mem_usage)


    sales = pd.read_csv(f"{INPUT_DIR}/sales_train_evaluation.csv",).pipe(
        reduce_mem_usage
    )
    submission = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv").pipe(
        reduce_mem_usage
    )

    print("sales shape:", sales.shape)
    print("prices shape:", prices.shape)
    print("calendar shape:", calendar.shape)
    print("submission shape:", submission.shape)


    return sales, prices, calendar, submission

In [ ]:
sales, prices, calendar, submission = read_data()

NUM_ITEMS = sales.shape[0]  # 30490
DAYS_PRED = submission.shape[1] - 1  # 28

In [ ]:
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df


calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales = encode_categorical(
    sales, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

prices = encode_categorical(prices, ["item_id", "store_id"]).pipe(reduce_mem_usage)

In [ ]:
def extract_num(ser):
    return ser.str.extract(r"(\d+)").astype(np.int16)


def reshape_sales(sales, submission, d_thresh=0, verbose=True):
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    evals_columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]
    # separate test dataframes.
    evals = submission[submission["id"].str.endswith("evaluation")]

    # get product table.
    
    
    product = sales[id_columns]
    sales = sales.melt(id_vars=id_columns, var_name="d", value_name="demand",)
    sales = reduce_mem_usage(sales)


    evals.columns = evals_columns#["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

    # merge with product table
    evals = evals.merge(product, how="left", on="id")


    if verbose:
        print("evaluation")
        display(evals)

    evals = evals.melt(id_vars=id_columns, var_name="d", value_name="demand")

    sales["part"] = "train"
    #sales["part"][~(sales["d"]<1914)] = "validation"
    evals["part"] = "evaluation"

    data = pd.concat([sales, evals], axis=0)

    del sales, evals

    data["d"] = extract_num(data["d"])
    data = data[data["d"] >= d_thresh]

    gc.collect()

    if verbose:
        print("data")
        display(data)

    return data


def merge_calendar(data, calendar):
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)
    return data.merge(calendar, how="left", on="d")


def merge_prices(data, prices):
    return data.merge(prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [ ]:
data = reshape_sales(sales, submission, d_thresh=1941 - int(365 * 2  + 7))
del sales
gc.collect()

calendar["d"] = extract_num(calendar["d"])
data = merge_calendar(data, calendar)
del calendar
gc.collect()

data = merge_prices(data, prices)
del prices
gc.collect()

data = reduce_mem_usage(data)

In [ ]:
def add_demand_features(df):
    for diff in [0, 56]:
        shift = DAYS_PRED + diff
        df[f"shift_t{shift}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(shift)
        )
    print("shift done")
    gc.collect()

    
    diff = 56
    for window in [56, 112, 168]:
        df[f"shift_t{diff}_rolling_std_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(diff).rolling(window).std()
        )
        df[f"shift_t{diff}_rolling_mean_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(diff).rolling(window).mean()
        )
        df[f"rolling_min_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(diff).rolling(window).min()
        )
        df[f"rolling_max_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(diff).rolling(window).max()
        )
        df[f"rolling_sum_t{window}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(diff).rolling(window).sum()
        )
    print("window done")
    df["rolling_skew_t56"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(28).skew()
    )
    df["rolling_kurt_t56"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(28).kurt()
    )
    return df


def add_price_features(df):
    df["shift_price_t1"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1)
    )
    df["price_change_t1"] = (df["shift_price_t1"] - df["sell_price"]) / (
        df["shift_price_t1"]
    )
    df["rolling_price_max_t365"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1).rolling(365).max()
    )
    df["price_change_t365"] = (df["rolling_price_max_t365"] - df["sell_price"]) / (
        df["rolling_price_max_t365"]
    )

    df["rolling_price_std_t7"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
    df["rolling_price_std_t30"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(30).std()
    )
    return df.drop(["rolling_price_max_t365", "shift_price_t1"], axis=1)


def add_time_features(df, dt_col):
    df[dt_col] = pd.to_datetime(df[dt_col])
    attrs = [
        "year",
        "quarter",
        "month",
        "week",
        "day",
        "dayofweek",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df[dt_col].dt, attr).astype(dtype)

    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    df['month_day']  = df['month'] * 100 + df['day']
    return df

In [ ]:
data = add_demand_features(data).pipe(reduce_mem_usage)
print('add_demand_features done')
data = add_price_features(data).pipe(reduce_mem_usage)
print('add_price_features done')
data = add_original_features(data).pipe(reduce_mem_usage)
print('add_original_features done')
dt_col = "date"
data = add_time_features(data, dt_col).pipe(reduce_mem_usage)
data = data.sort_values("date")

print("start date:", data[dt_col].min())
print("end date:", data[dt_col].max())
print("data shape:", data.shape)

### Training Model

In [ ]:
class CustomTimeSeriesSplitter:
    def __init__(self, n_splits=5, train_days=80, test_days=20, day_col="d"):
        self.n_splits = n_splits
        self.train_days = train_days
        self.test_days = test_days
        self.day_col = day_col

    def split(self, X, y=None, groups=None):
        SEC_IN_DAY = 3600 * 24
        sec = (X[self.day_col] - X[self.day_col].iloc[0]) * SEC_IN_DAY
        duration = sec.max()

        train_sec = self.train_days * SEC_IN_DAY
        test_sec = self.test_days * SEC_IN_DAY
        total_sec = test_sec + train_sec

        if self.n_splits == 1:
            train_start = duration - total_sec
            train_end = train_start + train_sec

            train_mask = (sec >= train_start) & (sec < train_end)
            test_mask = sec >= train_end

            yield sec[train_mask].index.values, sec[test_mask].index.values

        else:
            # step = (duration - total_sec) / (self.n_splits - 1)
            step = DAYS_PRED * SEC_IN_DAY

            for idx in range(self.n_splits):
                # train_start = idx * step
                shift = (self.n_splits - (idx + 1)) * step
                train_start = duration - total_sec - shift
                train_end = train_start + train_sec
                test_end = train_end + test_sec

                train_mask = (sec > train_start) & (sec <= train_end)

                if idx == self.n_splits - 1:
                    test_mask = sec > train_end
                else:
                    test_mask = (sec > train_end) & (sec <= test_end)

                yield sec[train_mask].index.values, sec[test_mask].index.values

    def get_n_splits(self):
        return self.n_splits

In [ ]:
day_col = "d"
cv_params = {
    "n_splits": 5,
    "train_days": 365,
    "test_days": 28,#DAYS_PRED,
    "day_col": day_col,
}
cv = CustomTimeSeriesSplitter(**cv_params)

In [ ]:
features = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "sell_price",
    # demand features
    "shift_t28",
    "shift_t56",
    # std
    "shift_t28_rolling_std_t28",
    "shift_t28_rolling_std_t56",
    "shift_t28_rolling_std_t84",
    # mean
    "shift_t28_rolling_mean_t28",
    "shift_t28_rolling_mean_t56",
    "shift_t28_rolling_mean_t84",
    # min,
    "rolling_min_t28",
    # max
    "rolling_max_t28",
    "rolling_max_t56",
    # sum
    "rolling_sum_t28",    
    "rolling_sum_t56",        
    "rolling_kurt_t28",    
    "price_change_t365",
    "rolling_price_std_t30",
    # time features
    "year",
    "quarter",
    "month",
    "week",
    "day",
    "dayofweek",
    "is_weekend",
    "month_day",
    # original features
    "shift_t28_log",
    "shift_t28_sqrt",
    "shift_t56_log",
    "shift_t56_sqrt",
    "shift_t28_diff_t7",
    'shift_t56_diff_t7'
    #"self_diff_t7",
    #"self_diff_t28"
    
]


is_train = (data["d"] < 1942)# & (data["d"]>=1544)

is_private = (data["d"] >= 1942)
is_public = ~(data["d"] < 1914) & ~(is_private)


# Attach "d" to X_train for cross validation.
X_train = data[is_train][[day_col] + features].reset_index(drop=True)
y_train = data[is_train]["demand"].reset_index(drop=True)
X_test_pub = data[is_public][features].reset_index(drop=True)
X_test_pri = data[is_private][features].reset_index(drop=True)

# keep these two columns to use later.
id_date_pub = data[is_public][["id", "date"]].reset_index(drop=True)
id_date_pri = data[is_private][["id", "date"]].reset_index(drop=True)

del data
gc.collect()

print(X_train['d'][0])
print("X_train shape:", X_train.shape)
print("X_test_pub shape:", X_test_pub.shape)
print("X_test_pri shape:", X_test_pri.shape)
print("id_date_pub shape:", id_date_pub.shape)
print("id_date_pri shape:", id_date_pri.shape)

In [ ]:
def train_lgb(bst_params, fit_params, X, y, cv, drop_when_train=None):
    models = []

    if drop_when_train is None:
        drop_when_train = []

    for idx_fold, (idx_trn, idx_val) in enumerate(cv.split(X, y)):
        print(f"\n----- Fold: ({idx_fold + 1} / {cv.get_n_splits()}) -----\n")

        X_trn, X_val = X.iloc[idx_trn], X.iloc[idx_val]
        y_trn, y_val = y.iloc[idx_trn], y.iloc[idx_val]
        
        print(f'\n train d min: {X_trn["d"].min()} \n valid d min: {X_val["d"].min()} \n')
        
        train_set = lgb.Dataset(
            X_trn.drop(drop_when_train, axis=1),
            label=y_trn,
            categorical_feature=["item_id"],
        )
        val_set = lgb.Dataset(
            X_val.drop(drop_when_train, axis=1),
            label=y_val,
            categorical_feature=["item_id"],
        )
        eval_result = {}
        model = lgb.train(
            bst_params,
            train_set,
            valid_sets=[train_set, val_set],
            valid_names=["train", "valid"],
            evals_result=eval_result,
            **fit_params,
        )
        models.append(model)

        del idx_trn, idx_val, X_trn, X_val, y_trn, y_val
        gc.collect()

    return models, eval_result

In [ ]:
bst_params =  {'lambda_l1': 0.00021301070633699974,
               'lambda_l2': 9.242591106708853e-07,
               'num_leaves': 31, 
               'feature_fraction': 0.584,
               'bagging_fraction': 1.0, 
               'bagging_freq': 0, 
               'min_child_samples': 20, 
               'boosting_type': 'gbdt',
               'metric': 'rmse',
               'objective': 'poisson',
               'n_jobs': -1,
               'seed': 42,
               'learning_rate': 0.03,
               'min_data_in_leaf': 20}

fit_params = {
    "num_boost_round": 100_000,
    "early_stopping_rounds": 100,
    "verbose_eval": 100,
}

models, evals = train_lgb(
    bst_params, fit_params, X_train, y_train, cv, drop_when_train=[day_col]
)

del X_train, y_train
gc.collect()

### prediction
Put the highest weight on the most recent timeseries model(fold 5) and put smaller weights on other folds.

In [ ]:
imp_type = "gain"
importances = np.zeros(X_test_pub.shape[1])
preds_pub = np.zeros(X_test_pub.shape[0])
preds_pri = np.zeros(X_test_pri.shape[0])

for model in models:
    importances += model.feature_importance(imp_type)

preds_pub = 0.6*models[5].predict(X_test_pub)+0.3*models[1].predict(X_test_pub)+0.1*models[4].predict(X_test_pub)
preds_pri = 0.6*models[5].predict(X_test_pri)+0.3*models[1].predict(X_test_pri)+0.1*models[4].predict(X_test_pri)
importances = importances / cv.get_n_splits()